In [1]:
import cv2
from cv2 import destroyAllWindows
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [3]:
def rescale_frame(frame, percent=50):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [4]:

angle_min = []
angle_min_hip = []
cap = cv2.VideoCapture("video1.mp4")



# Curl counter variables
counter = 0 
min_ang = 0
max_ang = 0
min_ang_hip = 0
max_ang_hip = 0
stage = None

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (640, 480)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output_video_.mp4', fourcc, 24, size)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if frame is not None:
            frame_ = rescale_frame(frame, percent=75)
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
           
        
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            angle_knee = calculate_angle(hip, knee, ankle) #Knee joint angle
            angle_knee = round(angle_knee,2)
            
            print("angle_knee:hip, knee, ankle")
            
            print(angle_knee)
            
            angle_hip = calculate_angle(shoulder, hip, knee)
            angle_hip = round(angle_hip,2)
            
            hip_angle = 180-angle_hip
            knee_angle = 180-angle_knee
            
            
            angle_min.append(angle_knee)
            angle_min_hip.append(angle_hip)
            
            #print(angle_knee)
            # Visualize angle
            """cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )"""
                       
                
            cv2.putText(image, str(angle_knee), 
                           tuple(np.multiply(knee, [1500, 800]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(angle_hip), 
                           tuple(np.multiply(hip, [1500, 800]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            # Curl counter logic
            if angle_knee > 169:
                stage = "up"
            if angle_knee <= 90 and stage =='up':
                stage="down"
                counter +=1
                print(counter)
                min_ang  =min(angle_min)
                max_ang = max(angle_min)
                
                min_ang_hip  =min(angle_min_hip)
                max_ang_hip = max(angle_min_hip)
                
                print(min(angle_min), " _ ", max(angle_min))
                print(min(angle_min_hip), " _ ", max(angle_min_hip))
                angle_min = []
                angle_min_hip = []
        except:
            pass
        
        # Render squat counter
        # Setup status box
        cv2.rectangle(image, (20,20), (435,160), (0,0,0), -1)
        
        
        
        cv2.putText(image, "angle_knee : " + str(angle_knee), 
                    (30,180), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        
        
        
        
        
        # Rep data
        """cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)"""
        cv2.putText(image, "Repetition : " + str(counter), 
                    (30,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        """cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)"""
        """cv2.putText(image, stage, 
                    (10,120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)"""
        
        #Knee angle:
        """cv2.putText(image, 'Angle', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)"""
        cv2.putText(image, "Knee-joint angle : " + str(min_ang), 
                    (30,100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        #Hip angle:
        cv2.putText(image, "Hip-joint angle : " + str(min_ang_hip), 
                    (30,140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        

        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(203,17,17), thickness=2, circle_radius=2) 
                                 )               
        
        out.write(image)
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            cap.release()
            out.release()
            cv2.destroyAllWindows()
            #break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

angle_knee:hip, knee, ankle
179.27
angle_knee:hip, knee, ankle
179.46
angle_knee:hip, knee, ankle
179.57
angle_knee:hip, knee, ankle
179.64
angle_knee:hip, knee, ankle
179.71
angle_knee:hip, knee, ankle
179.75
angle_knee:hip, knee, ankle
179.77
angle_knee:hip, knee, ankle
179.81
angle_knee:hip, knee, ankle
179.83
angle_knee:hip, knee, ankle
179.87
angle_knee:hip, knee, ankle
179.9
angle_knee:hip, knee, ankle
179.91
angle_knee:hip, knee, ankle
179.93
angle_knee:hip, knee, ankle
179.94
angle_knee:hip, knee, ankle
179.96
angle_knee:hip, knee, ankle
179.98
angle_knee:hip, knee, ankle
179.99
angle_knee:hip, knee, ankle
179.96
angle_knee:hip, knee, ankle
179.94
angle_knee:hip, knee, ankle
179.92
angle_knee:hip, knee, ankle
179.92
angle_knee:hip, knee, ankle
179.91
angle_knee:hip, knee, ankle
179.93
angle_knee:hip, knee, ankle
179.93
angle_knee:hip, knee, ankle
179.95
angle_knee:hip, knee, ankle
179.96
angle_knee:hip, knee, ankle
179.97
angle_knee:hip, knee, ankle
179.98
angle_knee:hip, knee,

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time

# Initialize Mediapipe Pose and Drawing modules
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    vec1 = np.array(a) - np.array(b)
    vec2 = np.array(c) - np.array(b)
    radians = np.arctan2(np.linalg.norm(np.cross(vec1, vec2)), np.dot(vec1, vec2))
    angle = np.degrees(radians)
    return angle

# def find_camera_index():
#     for i in range(20):  # You can adjust the range based on your system's available devices
#         cap = cv2.VideoCapture(i)
#         if cap.isOpened():
#             print(f"Camera index {i} is available.")
#             cap.release()
#         else:
#             print(f"Camera index {i} is not available.")

# find_camera_index()
            
# cap1 = cv2.VideoCapture(1)  # Use appropriate camera index here
# cap2 = cv2.VideoCapture(6) 



def find_available_cameras():
    available_cameras = []
    for i in range(10):  # Try indexing from 0 to 9 (adjust as needed)
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            available_cameras.append(i)
            cap.release()
    return available_cameras

# Example: Find available cameras and select the first two
available_cameras = find_available_cameras()
if len(available_cameras) >= 2:
    cap1 = cv2.VideoCapture(available_cameras[0])
    cap2 = cv2.VideoCapture(available_cameras[1])
    print(f"Capturing from cameras {available_cameras[0]} and {available_cameras[1]}")
else:
    print("Not enough cameras found.")
    

counter1 = 0
counter2 = 0

stage1 = None
stage2 = None

angle_values1 = []
angle_values2 = []

size1 = (640, 480)
size2 = (640, 480)

frame_rate1 = cap1.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out1 = cv2.VideoWriter('camera1_output.mp4', fourcc, 5, size1)

frame_rate2 = cap2.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out2 = cv2.VideoWriter('camera2_output.mp4', fourcc, 5, size2)

duration = 10  # 10 seconds

start_time = time.time()

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap1.isOpened() and cap2.isOpened():
        elapsed_time = time.time() - start_time
        current_time = time.time() - start_time
        
#         if elapsed_time >= duration:
#             print("Time period elapsed. Stopping the loop.")
#             break
            
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        
        if not ret1 or not ret2:
            break
        
        image1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
        image1.flags.writeable = False
        results1 = pose.process(image1)
        image1.flags.writeable = True
        image1 = cv2.cvtColor(image1, cv2.COLOR_RGB2BGR)
        
        image2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
        image2.flags.writeable = False
        results2 = pose.process(image2)
        image2.flags.writeable = True
        image2 = cv2.cvtColor(image2, cv2.COLOR_RGB2BGR)
        
        
        
        try:
            landmarks1 = results1.pose_landmarks.landmark
            shoulder1 = [landmarks1[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow1 = [landmarks1[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist1 = [landmarks1[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            hip1 = [landmarks1[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee1 = [landmarks1[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle1 = [landmarks1[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks1[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            angle_knee1 = calculate_angle(hip1, knee1, ankle1)
            
            angle_hip1 = calculate_angle(shoulder1, elbow1, wrist1)
            hip_angle1 = 180 - angle_hip1
            
            knee_angle1 = 180 - angle_knee1

            # Visualization and counter logic
            cv2.putText(image1, str(hip_angle1),
                        tuple(np.multiply(knee1, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (79, 121, 66), 2, cv2.LINE_AA)

            if angle_knee1 > 169:
                stage1 = "UP"
            if angle_knee1 <= 90 and stage1 == 'UP':
                stage1 = "DOWN"
                counter1 += 1
                print(counter1)

            # Drawing landmarks and connections
            mp_drawing.draw_landmarks(image1, results1.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            out1.write(image1)
            angle_values1.append({'Time': current_time, 'Camera 1 Angle': hip_angle1})
            
            landmarks2 = results2.pose_landmarks.landmark

            shoulder2 = [landmarks2[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks2[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow2 = [landmarks2[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks2[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist2 = [landmarks2[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks2[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            hip2 = [landmarks2[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks2[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee2 = [landmarks2[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks2[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle2 = [landmarks2[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks2[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            angle_knee2 = calculate_angle(hip2, knee2, ankle2)
            
            angle_hip2 = calculate_angle(shoulder2, elbow2, wrist2)
            hip_angle2 = 180 - angle_hip2
            
            knee_angle2 = 180 - angle_knee2

            # Visualization and counter logic
            cv2.putText(image2, str(hip_angle2),
                        tuple(np.multiply(knee2, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (79, 121, 66), 2, cv2.LINE_AA)

            if angle_knee2 > 169:
                stage2 = "UP"
            if angle_knee2 <= 90 and stage2 == 'UP':
                stage2 = "DOWN"
                counter2 += 1
                print(counter2)

            # Drawing landmarks and connections
            mp_drawing.draw_landmarks(image2, results2.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            out2.write(image2)
            angle_values2.append({'Time': current_time, 'Camera 2 Angle': hip_angle2})
               
                
        except Exception as e:
            print("Error:", e)
            
        cv2.imshow('Camera 1', image1)
        cv2.imshow('Camera 2', image2)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap1.release()
cap2.release()
cv2.destroyAllWindows()

# Create DataFrames from the angle lists
df1 = pd.DataFrame(angle_values1)
df2 = pd.DataFrame(angle_values2)

try:
    df1.to_excel('camera1_angles.xlsx', index=False)
    df2.to_excel('camera2_angles.xlsx', index=False)
    print("Excel files saved successfully.")
except Exception as e:
    print("Error saving Excel files:", e)

try:
    out1.release()
    out2.release()
    print("Video files saved successfully.")
except Exception as e:
    print("Error saving Video files:", e)

Capturing from cameras 0 and 1
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Excel files saved successfully.
Video files saved successfully.
